<h1>Desenvolver um programa para verificar a nota do aluno em uma prova com 10 questões, <br>
o programa deve comparar a resposta de cada questão com o gabarito da prova e assim calcular<br>
o total de acertos e a nota (atribuir 1 ponto por resposta certa). </h1>
<br>
<h2>Após todas as notas serem calculadas, o sistema deve salvar em arquivo:</h2>
<br>
<h3><p>Maior e Menor Acerto;</p>
<p>Total de aprovados;</p>
<p>A Média das Notas da Turma.</p>
<p>Arquivos contidos na pasta "correcao" >> gabarito.csv, provas.csv, notas.csv, relatorio.csv</p></h1>

In [92]:
import csv

In [93]:
def importar_dados(endereco):
    with open(endereco, "r") as f:
        arquivo = csv.reader(f, delimiter="\t")
        lista = []
        for indice, linha in enumerate(arquivo):
            if indice > 0:
                lista.append(tuple(linha))
        return tuple(lista)

In [94]:
def lista_para_dicionario(lista):
    dicio = {}
    for l in lista:
        dicio[l[0]] = l[1:]
    return dicio

In [95]:
def calcula_nota(dicionario, correcao):
    #Aqui temos uma nested function, ou seja, uma função dentro de outra. Ela não pode ser usada fora da função externa
    #mas pode utilizar as variáveis locais da função
    def corrigir(respostas): 
        gabarito = correcao[1:]
        cont = 0
        for indice, valor in enumerate(respostas):
            if valor == gabarito[indice]:
                cont += 1
        return cont
    
    notas = []
    for chave, valor in dicionario.items():
        nota = corrigir(valor)
        situacao = aprovar(nota, 6)
        notas.append({"Matricula": chave, "Nota": nota, "Situação": situacao})
    return notas

In [96]:
def menor(lista):
    minimo = 10
    saida = 0
    for _, nota in enumerate(lista):
        if nota["Nota"] < minimo:
            minimo = nota["Nota"]
            saida = (minimo, nota["Matricula"])
    return saida

In [97]:
def maior(lista):
    maximo = 0
    saida = 0
    for _, nota in enumerate(lista):
        if nota["Nota"] > maximo:
            maximo = nota["Nota"]
            saida = (maximo, nota["Matricula"])
    return saida

In [111]:
def media_notas(lista):
    soma = 0
    for _, nota in enumerate(lista):
        soma += nota["Nota"]
    return round(soma/(len(lista)),3)

In [99]:
def aprovar(nota, media):
    if nota >= media:
        return "Aprovado"
    elif nota >= media*0.4:
        return "Recuperação"
    else:
        return "Reprovado"

In [100]:
def conte_se(origem, condicao, valor):
    return len([x for x in origem if x[condicao] == valor])

In [101]:
def salvar_informacao(local, arquivo, mode = "w+"):
    with open(local, mode) as f:
        try:
            header = [x for x in arquivo.keys()]
            
        except AttributeError:
            header = [x for x in arquivo[0].keys()]
        finally:
            gravar = csv.DictWriter(f, delimiter = "\t", fieldnames = header)
            gravar.writeheader()
            try:
                gravar.writerows(arquivo)
            except AttributeError:
                gravar.writerow(arquivo)

In [135]:
def main():
    gabarito = importar_dados("correcao/gabarito.csv")[0]
    lista_provas = importar_dados("correcao/provas.csv")
    dicionario_provas = lista_para_dicionario(lista_provas)
    notas = calcula_nota(dicionario_provas, gabarito)

    maior_nota, aluno_maior_nota = maior(notas)
    menor_nota, aluno_menor_nota = menor(notas)
    media = media_notas(notas)       #ok
    total_aprovados = conte_se(notas, "Situação", "Aprovado")
    total = len(notas)
    percentagem = " "*25 + str(round(total_aprovados/total * 100, 2)) + "%"
    
    #maior_nota = f"\t{maior_nota}"
    #menor_nota = f"\t{menor_nota}"
    #media = f"{media:.2f}"
    total_aprovados = "       " + str(total_aprovados)
    total = " "*20 + str(total)
    
    info = {"ID >": aluno_maior_nota, "Maior nota": maior_nota, 
            "ID <": aluno_menor_nota, "Menor nota": menor_nota, 
            "Média": media, "Aprovados": total_aprovados, "Total de alunos": total, "Taxa de Aprovação": percentagem}


    salvar_informacao("correcao/notas.csv", notas)
    salvar_informacao("correcao/notas.csv", [{" ": " "}], mode = "a")
    salvar_informacao("correcao/notas.csv", info, mode = "a")
    salvar_informacao("correcao/relatorio.csv", info)
    print("Dados gerados e salvos com sucesso!!")

In [136]:
main()

Dados gerados e salvos com sucesso!!
